In [1]:
from scipy import stats
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lazypredict
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import GridSearchCV



Đọc và chia data

In [2]:
data = pd.read_csv("DataAfterCleaning.csv", delimiter=",")
data.head()

,chieuDai,chieuNgang,dienTich,Gia/m2,Phongngu,SoTang,PhongTam,Loai,GiayTo,TinhTrangNoiThat,QuanHuyen
0,20.00,5.00,100.00,604.17,3,1.00,3.00,"nhà ngõ, hẻm",đã có sổ,nội thất đầy đủ,quận 8
1,16.00,5.00,80.00,1718.75,2,1.00,2.00,"nhà ngõ, hẻm",đã có sổ,NaN,quận 12
2,15.00,8.00,117.40,4542.92,8,5.00,7.00,nhà biệt thự,đã có sổ,hoàn thiện cơ bản,quận gò vấp
3,32.00,3.20,102.00,2982.08,3,2.00,2.00,"nhà mặt phố, mặt tiền",đã có sổ,nội thất đầy đủ,quận tân phú
4,4.00,6.00,26.00,4295.00,2,2.00,3.00,"nhà ngõ, hẻm",đã có sổ,nội thất đầy đủ,quận phú nhuận


In [3]:
data.SoTang.unique()

array([ 1.,  5.,  2.,  3.,  4., 11.,  6., 22., 23., 12.,  7., 13., 15.,
        9.,  8.])

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13432 entries, 0 to 13431
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   chieuDai          13432 non-null  float64
 1   chieuNgang        13432 non-null  float64
 2   dienTich          13432 non-null  float64
 3   Gia/m2            13432 non-null  float64
 4   Phongngu          13432 non-null  int64  
 5   SoTang            13432 non-null  float64
 6   PhongTam          13432 non-null  float64
 7   Loai              13431 non-null  object 
 8   GiayTo            13408 non-null  object 
 9   TinhTrangNoiThat  9001 non-null   object 
 10  QuanHuyen         13432 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 1.1+ MB


In [5]:
target = "Gia/m2"
x = data.drop(target, axis=1)
y = data[target]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=1609)

In [6]:
print("train.shape: ",x_train.shape)
print("test.shape: ",x_test.shape)

train.shape:  (10745, 10)
test.shape:  (2687, 10)


Xem unique của từng cột categorical

In [7]:
print("uniqueLoai:")
x_train.Loai.unique()

uniqueLoai:


array(['nhà ngõ, hẻm', 'nhà mặt phố, mặt tiền', 'nhà phố liền kề',
       'nhà biệt thự', nan], dtype=object)

In [8]:
print("uniqueGiayTo:")
x_train.GiayTo.unique()

uniqueGiayTo:


array(['đã có sổ', 'giấy tờ viết tay', 'không có sổ', 'đang chờ sổ',
       'sổ chung / công chứng vi bằng', nan], dtype=object)

In [9]:
print("uniqueTinhTrangNoiThat:")
x_train.TinhTrangNoiThat.unique()

uniqueTinhTrangNoiThat:


array(['nội thất đầy đủ', 'bàn giao thô', nan, 'hoàn thiện cơ bản',
       'nội thất cao cấp'], dtype=object)

In [10]:
print("uniqueQuanHuyen:")
x_train.QuanHuyen.unique()

uniqueQuanHuyen:


array(['quận 12', 'huyện bình chánh', 'quận 6', 'quận 1', 'quận 10',
       'quận gò vấp', 'quận bình tân', 'quận phú nhuận', 'quận 8',
       'quận 7', 'quận tân bình', 'huyện nhà bè', 'huyện hóc môn',
       'quận bình thạnh', 'quận 11', 'quận 5', 'quận tân phú', 'quận 3',
       'quận 4', 'huyện củ chi', 'huyện cần giờ'], dtype=object)

In [11]:
## lấy ra list quận huyện theo thứ tự tăng dần của giá
df = pd.DataFrame(data)

def sorted_districts(col):
    table = df.groupby(col)['Gia/m2'].mean().reset_index()
  
    sorted_table = table.sort_values(by='Gia/m2')
    sorted_districts = sorted_table[col].tolist()
    return sorted_districts

list_sorted_districts = sorted_districts('QuanHuyen')

In [12]:
for i in range(0, len(list_sorted_districts), 4):
    print(list_sorted_districts[i:i+4])

['huyện cần giờ', 'huyện củ chi', 'huyện bình chánh', 'huyện hóc môn']
['huyện nhà bè', 'quận 12', 'quận bình tân', 'quận 8']
['quận tân phú', 'quận gò vấp', 'quận 7', 'quận 6']
['quận bình thạnh', 'quận 4', 'quận tân bình', 'quận 11']
['quận phú nhuận', 'quận 10', 'quận 5', 'quận 3']
['quận 1']


Tiền xử lý lần 2

In [13]:
Loai_values = ["unknown",'nhà ngõ, hẻm', 'nhà phố liền kề', 'nhà mặt phố, mặt tiền', 'nhà biệt thự']
GiayTo_values = ["unknown",'giấy tờ viết tay', 'không có sổ', 'đang chờ sổ', 
            'sổ chung / công chứng vi bằng', 'đã có sổ']
TinhTrangNoiThat_values = ["unknown",'bàn giao thô', 'hoàn thiện cơ bản', 'nội thất đầy đủ', 'nội thất cao cấp']
QuanHuyen_values = list_sorted_districts

ord_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="unknown")),
    ("encoder", OrdinalEncoder(categories=[Loai_values,GiayTo_values,TinhTrangNoiThat_values,QuanHuyen_values]))
])

num_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
preprocessor = ColumnTransformer(transformers=[
    ("num_features", num_transformer, ["chieuDai", "chieuNgang","dienTich","Phongngu","SoTang","PhongTam"]),
    ("ordinal_features", ord_transformer, ["Loai","GiayTo","TinhTrangNoiThat","QuanHuyen"]),
])



In [14]:
x_train_preprocessor = preprocessor.fit_transform(x_train)
x_test_preprocessor = preprocessor.transform(x_test)
lazy_reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None )
models, predictions = lazy_reg.fit(x_train_preprocessor, x_test_preprocessor, y_train, y_test)
print(predictions)

100%|██████████| 42/42 [00:54<00:00,  1.30s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000647 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 785
[LightGBM] [Info] Number of data points in the train set: 10745, number of used features: 10
[LightGBM] [Info] Start training from score 4447.933647
                               Adjusted R-Squared  R-Squared      RMSE  \
Model                                                                    
LGBMRegressor                                0.71       0.71   1547.59   
XGBRegressor                                 0.71       0.71   1553.67   
HistGradientBoostingRegressor                0.71       0.71   1554.45   
RandomForestRegressor                        0.70       0.70   1589.31   
GradientBoostingRegressor                    0.69       0.69   1598.82   
ExtraTreesRegressor                          0.67       0.67   1652.69   


In [15]:
from sklearn.linear_model import PoissonRegressor,LinearRegression
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
reg = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor",RandomForestRegressor()),
])
# reg.fit(x_train, y_train)

In [ ]:
parameters = {
    "regressor__n_estimators": [50, 100, 200, 500],
    "regressor__criterion": ["squared_error", "absolute_error", "poisson"],
    "regressor__max_depth": [None, 5, 10, 20],
    "regressor__max_features": ["sqrt", "log2"],
    "preprocessor__num_features__imputer__strategy": ["mean", "median"],
}
model = GridSearchCV(reg, param_grid=parameters, scoring="r2", cv=6, verbose=2, n_jobs=-1)
model.fit(x_train, y_train)
print(model.best_score_)
print(model.best_params_)

In [41]:
y_predict = model.predict(x_test)

In [ ]:
accuracy_list = [(i, j, abs(i - j) / i * 100 if i != 0 else 0) for i, j in zip(y_test, y_predict)]

accuracy_list_sorted = sorted(accuracy_list, key=lambda x: x[2])

for actual, predict, percentage_error in accuracy_list_sorted:
    print("Actual: {} - Predict: {} - Percentage Error: {:.2f}%".format(actual, predict, percentage_error))



In [ ]:
print("MAE {}".format(mean_absolute_error(y_test, y_predict)))
print("MSE {}".format(mean_squared_error(y_test, y_predict)))
print("R2 {}".format(r2_score (y_test, y_predict)))